In [3]:
import pandas as pd
from transformers import MBartForConditionalGeneration, AlbertTokenizer, Trainer, TrainingArguments
from datasets import Dataset

2024-05-01 18:49:11.663299: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 18:49:11.663398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 18:49:11.787218: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**Initialize the model and tokenizer**

In [4]:
model = MBartForConditionalGeneration.from_pretrained("ai4bharat/IndicBART")
tokenizer = AlbertTokenizer.from_pretrained("ai4bharat/IndicBART", do_lower_case=False, use_fast=False, keep_accents=True)

config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/976M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Install required libraries for devenagari conversion**

In [6]:
!pip install indic_transliteration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.1/152.1 kB 5.9 MB/s eta 0:00:00


**Loading the train dataset and telugu to devanagari conversion**

In [7]:
from datasets import Dataset
import pandas as pd
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

# Read the CSV file into a DataFrame
df = pd.read_csv("/kaggle/input/final-dataset/final_train.csv")

# Function to transliterate Telugu text to Devanagari script
def telugu_to_devanagari(text):
    return transliterate(text, sanscript.TELUGU, sanscript.DEVANAGARI)

# Apply the transliteration function to 'Text' and 'Summary' columns

df["Text"] = df["Text"].astype(str) 
df["Summary"] = df["Summary"].astype(str)
df['Text_Devanagari'] = df['Text'].apply(telugu_to_devanagari)
df['Summary_Devanagari'] = df['Summary'].apply(telugu_to_devanagari)

# Save the DataFrame to a new CSV file
#df.to_csv("devanagari_converted.csv", index=False)

# Create a dictionary from DataFrame
dataset_dict = {"text": df["Text"].tolist(), "summary": df["Summary"].tolist()}

# Create Hugging Face Dataset
dataset = Dataset.from_dict(dataset_dict)


**Defining training arguments**

In [8]:
training_args = TrainingArguments(
    output_dir="./results",  # output directory
    num_train_epochs=5,  # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    save_steps=100000,  # number of updates steps before checkpoint saves
    save_total_limit=2,  # limit the total amount of checkpoints
)

**Defining function to preprocess the dataset**

In [9]:
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], return_tensors="pt", max_length=512, padding='max_length', truncation=True)
    targets = tokenizer(examples["summary"], return_tensors="pt", max_length=128, padding='max_length', truncation=True)
    return {"input_ids": inputs.input_ids, "attention_mask": inputs.attention_mask, "labels": targets.input_ids}

**Preprocessing the dataset**

In [10]:
train_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10414 [00:00<?, ? examples/s]

**Initializing the trainer**

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    #compute_metrics=compute_metrics,
)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


**Finetuning the model**

In [12]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.378300
1000,0.374200
1500,0.363000
2000,0.377800
2500,0.365500
3000,0.370400
3500,0.363500
4000,0.384700
4500,0.359300
5000,0.358500


TrainOutput(global_step=13020, training_loss=0.3573411896115258, metrics={'train_runtime': 4475.1718, 'train_samples_per_second': 11.635, 'train_steps_per_second': 2.909, 'total_flos': 2.821126694633472e+16, 'train_loss': 0.3573411896115258, 'epoch': 5.0})

**Loading the test dataset**

In [ ]:
from datasets import Dataset
import pandas as pd

# Load CSV dataset
df2 = pd.read_csv("/kaggle/input/article-headline/cftest.csv")

# Ensure "Text" and "Summary" columns are string type
df2["Text"] = df2["Text"].astype(str) 
df2["Summary"] = df2["Summary"].astype(str)

# Create a dictionary from DataFrame
dataset_dict2 = {"text": df2["Text"].tolist(), "summary": df2["Summary"].tolist()}

# Create Hugging Face Dataset
dataset2 = Dataset.from_dict(dataset_dict2)

**Telugu to Devenagari conversion**

In [15]:
from datasets import Dataset
import pandas as pd
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

# Read the CSV file into a DataFrame
df2 = pd.read_csv("/kaggle/input/final-dataset/final_test.csv")

# Function to transliterate Telugu text to Devanagari script
def telugu_to_devanagari(text):
    return transliterate(text, sanscript.TELUGU, sanscript.DEVANAGARI)

# Apply the transliteration function to 'Text' and 'Summary' columns

df2["Text"] = df2["Text"].astype(str) 
df2["Summary"] = df2["Summary"].astype(str)
df2['Text_Devanagari'] = df2['Text'].apply(telugu_to_devanagari)
df2['Summary_Devanagari'] = df2['Summary'].apply(telugu_to_devanagari)

# Save the DataFrame to a new CSV file
#df.to_csv("devanagari_converted.csv", index=False)

# Create a dictionary from DataFrame
dataset_dict2 = {"text": df2["Text"].tolist(), "summary": df2["Summary"].tolist()}

# Create Hugging Face Dataset
dataset2 = Dataset.from_dict(dataset_dict2)


**Preprocessing the test dataset**

In [16]:
test_dataset = dataset2.map(preprocess_function, batched=True)

Map:   0%|          | 0/2604 [00:00<?, ? examples/s]

**Generating the summaries**

In [17]:
import torch

# List to store generated summaries
generated_summaries = []

# Move model to appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the maximum length for the generated summaries
max_length = 128  # Adjust this value as needed

# Iterate over the tokenized test dataset
count = 0
for example in test_dataset:
    # Convert input_ids and attention_mask to tensors and move to appropriate device
    input_ids = torch.tensor([example["input_ids"]]).to(device)
    attention_mask = torch.tensor([example["attention_mask"]]).to(device)
    
    # Generate prediction for the current example
    prediction = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_length)
    
    # Decode the prediction and append it to the list
    generated_summary = tokenizer.decode(prediction[0], skip_special_tokens=True)
    generated_summaries.append(generated_summary)
    count += 1
    #if count > 1000:
        #break


**Devanagari to Telugu conversion**

In [18]:
from indic_transliteration import sanscript

# Define a function to convert Devanagari script to Telugu script
def devanagari_to_telugu(text):
    return sanscript.transliterate(text, sanscript.DEVANAGARI, sanscript.TELUGU)

# List to store generated summaries in Telugu script
generated_summaries_telugu = []

# Iterate over the generated summaries in Devanagari script
for summary_devanagari in generated_summaries:
    # Convert each summary to Telugu script and append to the list
    summary_telugu = devanagari_to_telugu(summary_devanagari)
    generated_summaries_telugu.append(summary_telugu)

**Printing the summaries**

In [19]:
for i in range(10):
    print(f"Text: {df2['Text'][i]}")
    print(f"Original Summary: {df2['Summary'][i]}")
    print(f"Generated Summary: {generated_summaries_telugu[i]}\n")
    #print(f"Generated Summary: {generated_summaries_telugu[i]}\n")

Text: ఖమ్మంక్రైం: వీరిద్దరే కాదు జిల్లాలో ఎన్నికల కోడ్‌ అమల్లోకి వచ్చిన నాటి నుంచి వ్యాపారులు, చాలా మంది ప్రజలు ఈ పరిస్థితి ఎదుర్కొంటున్నారు. అవసరాలు, కారణాలు సరిగ్గా పరిశీలించకుండా.. వివరాలు తెలుసుకోకుండా కనిపించిన నగదును సీజ్‌ చేస్తుండంతో ఇబ్బంది పడుతున్నారు. బ్యాంకులు, వ్యాపార కూడళ్ల వద్ద మఫ్టీలో ఉంటున్న పోలీసులు రూ.50 వేలకు మించి ఎవరి వద్ద నగదు ఉన్నా స్వాధీనం చేసుకుంటున్నారు. అన్ని పత్రాలు ఉండాలని అధికారులు చెబుతున్నా.. రోజువారీ లావాదేవీలు, వ్యాపార, కుటుంబ అవసరాల కోసం నగదు తీసుకెళ్తున్న వారు చిక్కుల్లో పడుతున్నారు. బ్యాంకుల వద్ద మకాం ఎన్నికల కోడ్‌ అమలులోకి వచ్చిన దగ్గర నుంచి చెక్‌పోస్ట్‌ల వద్ద తనిఖీలు ముమ్మరం చేసిన పోలీసులు.. గతంలో ఎన్నడూ లేని విధంగా బ్యాంకులు, వ్యాపార లావాదేవీలు జరిగే ప్రాంతంలో తిష్ట వేయడం గమనార్హం. నిత్యం వ్యాపారాలు చేసే వారికి రూ.లక్షల్లో నగదు లావాదేవీలు తప్పనిసరి. ఇంకోపక్క వివాహాల సీజన్‌ కావడంతో దుస్తులు, వెండి, బంగారం కొనుగోలుకు వస్తున్న వారు సైతం పోలీసుల చేతుల్లో చిక్కుకుంటున్నారు. దీంతో పోలీసులను చూస్తే భయపడాల్సిన పరిస్థితులు ఎదురవుతున్నాయి. ఒకవేళ ఎన్నికల కో

**Function to calculate BLEU score**

In [20]:
from nltk.translate.bleu_score import sentence_bleu

def evaluate_summary(reference, generated):
    # BLEU score
    bleu_score = sentence_bleu([reference], generated)

    # ROUGE score

    return bleu_score


**Avg BLEU score calculation**

In [ ]:
avg_bleu = 0
bleu = 0
for i in range(1000):
    #print(f"Text: {df2['Text'][i]}")
    #print(f"Original Summary: {df2['Summary'][i]}")
    reference_summary = df2["Summary"][i]
    #print(f"Generated Summary: {generated_summaries[i]}\n")
    generated_summary = generated_summaries[i]
    bleu = evaluate_summary(reference_summary, generated_summary)
    avg_bleu += bleu
print(avg_bleu/1000)

In [21]:
avg_bleu = 0
bleu = 0
for i in range(len(df2["Summary"])):
    #print(f"Text: {df2['Text'][i]}")
    #print(f"Original Summary: {df2['Summary'][i]}")
    reference_summary = df2["Summary"][i]
    #print(f"Generated Summary: {generated_summaries[i]}\n")
    generated_summary = generated_summaries_telugu[i]
    bleu = evaluate_summary(reference_summary, generated_summary)
    avg_bleu += bleu
print(avg_bleu/len(df2["Summary"]))

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.2980717400945589


**Storing the generated summaries in a csv file**

In [22]:
import csv

# Assuming you have a list of generated summaries called generated_summaries
#generated_summaries = ['Summary 1', 'Summary 2', 'Summary 3', ...]

# Specify the file name
csv_file = '/kaggle/working/indicbart_generated_summaries1.csv'  # Saving in the working directory in Kaggle

# Open the CSV file in write mode and write the summaries
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    # Create a CSV writer object
    writer = csv.writer(file)
    
    # Write each summary in the list as a row with a single column
    for summary in generated_summaries_telugu:
        writer.writerow([summary])

print("File created and data stored successfully.")

File created and data stored successfully.


In [24]:
!zip -r 1indicbart_latest_gensum1.zip /kaggle/working/indicbart_generated_summaries1.csv

  adding: kaggle/working/indicbart_generated_summaries1.csv (deflated 78%)


In [25]:
from IPython.display import FileLink
FileLink(r'1indicbart_latest_gensum1.zip')

/kaggle/working/1indicbart_latest_gensum1.zip